## Prelude - Gridworld

For the rest of the course we will be focussing on a game we call Gridworld. In many ways this is simpler than the tic-tac-toe game investigated in last chapter.

<img src="figures/03 - Gridworld.png" alt="Gridworld" style="width: 200px;"/>

The above is a visual representation of the environment. There are only eleven states and four actions (up, down, left, right) in this game, but it will allow us to explore the main features of RL.

# The Markov Property

Given a sequence of RVs
$$ \left\lbrace x_1, x_2, \dotsc, x_t \right\rbrace $$
we can find the probability
$$ p \left\lbrace x_t \mid x_{t-1}, x_{t-2}, \dotsc, x_1 \right\rbrace .$$
Generally, the above can't be simplified.

The Markov Property specifies how many previous terms the current term depends on. The \[first-order\] Markov Property states that
$$p \left\lbrace x_t \mid x_{t-1}, x_{t-2}, \dotsc, x_1 \right\rbrace = p \left\lbrace x_t \mid x_{t-1} \right\rbrace ,$$
the second-order Markov Property states that
$$p \left\lbrace x_t \mid x_{t-1}, x_{t-2}, \dotsc, x_1 \right\rbrace = p \left\lbrace x_t \mid x_{t-1}, x_{t-2} \right\rbrace ,$$
etc.

### Application to RL

When used in the context of RL, the Markov Property states that
$$p \left\lbrace S_{t+1}, R_{t+1} \mid S_t, A_t, S_{t-1}, A_{t-1}, \dotsc, S_0, A_0  \right\rbrace = p \left\lbrace S_{t+1}, R_{t+1} \mid S_t, A_t  \right\rbrace  .$$
This is written in notation as
$$ p(s', r \mid s,a) = p \left\lbrace S_{t+1}, R_{t+1} \mid S_t, A_t \right \rbrace  .$$

This is the joint probability of $s'$ and $r$, conditioned on two other variables (whereas the "usual" Markov Property is just one RV conditioned on one other RV).

### Related Conditional Probabilities

Other useful conditional distributions (which can be directly derived from rules of probability) are
$$ p (s' \mid s,a) = \sum_{r \in R} p (s', r \mid s, a)  ,$$
$$ p (r \mid s,a) = \sum_{s' \in S} p (s', r \mid s, a)  .$$

For this course and most cases, the above equations will be deterministic. This is not always the case though (eg. partially observable games).

# Markov Decision Processes

A Markov Decision Process is a 5-tuple of:
<ul>
    <li>The set of states, $\lbrace s_i \rbrace$</li>
    <li>The set of actions, $\lbrace a_i \rbrace$</li>
    <li>The set of rewards, $\lbrace r_i \rbrace$</li>
    <li>The state-transition probabilities, $\lbrace  p(s' \mid s,a) \rbrace$</li>
    <li>The discount factor, $\gamma$.</li>
</ul>

The other key term in the MDP is the policy, $\pi$. While not explicitly part of the MDP itself, it helps guide actions along with the value function. There is no explicit equation for finding the policy (aside from the optimal policy which is defined in terms of the value function) - the policy is more like an algorithm, like decaying epsilon-greedy.

# The Discount Factor

The value of a reward, in real may diminish over time (eg. money). This is sort-of the case in RL. The larger the number of steps an agent looks into the future, the more blurry/inaccurate the predictions get.

We call the total future reward the return, $G(t)$, defined as
$$G(t) = \sum_{\tau = 0}^\infty \gamma^\tau R(t + 1 + \tau) $$
where $\gamma$ is the Discount Factor.

Usually, the Discount Factor is close to $1$, and for short episodic tasks, it may even be $1$.

## Merging Episiodic and Continuous Tasks

The fact that the sum goes to infinity makes the maths easier. However, this implies that the task never ends, and hence is continuous. The way to make them equivalent is to make the terminal state infinitely loop back to itself after the episode is "over".

# Value Functions (formal definition inc. derivation of Belmann's Eqn)

The Value Function, given a policy $\pi$ and a state $s$, is equal to the expected value of the return given that you are currently in state $s$:
$$V_\pi (s) = E_\pi \left[ G(t) \mid s \right] = E_\pi \left[ \left( \sum_{\tau = 0}^\infty \gamma^\tau R\left( t + 1 + \tau \right) \right) \mid s \right]  .$$

Note that the Value Function depends on <b>future</b> rewards, so the Value Function at the terminal state always equals $0$.

### Recursiveness

Note that the Value Function is recursive as
$$\sum_{\tau = 0}^\infty \gamma^\tau R(t+1+\tau) = \gamma^0 R(t+1) + \sum_{\tau=}^\infty \gamma^{\tau+1} R(t+2+\tau) = R(t+1) + \gamma \sum_{\tau}^\infty \gamma^{\tau} R(t+2+\tau)  ,$$

thus
$$V_\pi (s) = E_\pi \left[ \left( R(t+1) + \gamma \sum_{\tau}^\infty \gamma^{\tau} R(t+2+\tau) \right) \mid s \right]  ,$$

and so
$$V_\pi (s) = E_\pi \left[ \left( R(t+1) + \gamma G(t+1) \right) \mid s \right]  .$$

### The Policy and Investigation of V(s)

$V_\pi (s)$ is the expected value over $\pi$, so we are able to express it as a probability distribution, namely $\pi = \pi(a \mid s)$. This is actually the mathematical definition of the policy $\pi$.

As the expectation is a linear operator, we are able to investigate $V_\pi (s)$ in parts. The first part we investigate is $V_\pi (s) = E_\pi \left[ R(t+1) \mid s \right] $. This is the expectation of $R$ given $s$. However, note that $R$ depends on $r,a,s',s$. $s$ is the only given, so we sum over all other variables (dividing by the reward as usual in expectation) as follows

$$\sum_r \sum_{s'} \sum_a p(r,a,s' \mid s) r  .$$

Directly from the chain rule in probability ($p(x,y \mid z) = p(y \mid z) p(x \mid y,z)$), we know that 

$$p(r,a,s' \mid s) = p( a \mid s) p(r, s' \mid s,a)  .$$

The chain rule we used is derived as follows:

$$p(x,y \mid z) = \dfrac{p(x,y,z)}{p(z)} = \dfrac{p(x \mid y,z) p(y,z)}{p(z)} = p(x \mid y,z) p(y \mid z)  .$$

So we actually have

$$E_\pi \left[ R(t+1) \mid s \right] = \sum_r \sum_{s'} \sum_a p(r,a,s' \mid s) r = \sum_r \sum_{s'} \sum_a p( a \mid s) p(r, s' \mid s,a) r  .$$

However, we can go further. Because we're not really interested in $r$ for specific new states $s'$, but $r$ in general (regardless of the new state), we can marginalise out $s'$. This gives us

$$E_\pi \left[ R(t+1) \mid s \right] = \sum_r \sum_a p( a \mid s) p(r \mid s,a) r  ,$$

which is rearranged as (for convenience):

$$E_\pi \left[ R(t+1) \mid s \right] = \sum_a \left( p( a \mid s) \sum_r p(r \mid s,a) r \right) = \sum_a \left( \pi( a \mid s) \sum_r r p(r \mid s,a) \right)  .$$

### Some Comments

<ul>
    <li> Note that this expression is actually quite general:
$$E_\pi \left[ (\text{anything}) \mid s \right] = \sum_a \left( \pi( a \mid s) \sum_r \sum_{s'} (\text{anything}) p(r,s' \mid s,a) \right)  .$$ </li>
    <li> The splitting up of probability distributions has some intuition: $\pi(s \mid a)$ comes from "decisions being made by the agent, whereas $p(r, s' \mid s, a)$ comes from the environment. </li>
    <li> $r$ can sometimes be thought of as $r(s')$ - it's the reward you get for arriving in state $s'$, in which case you can also sum over $s'$. </li>
</ul>


### Continued Investigation of V(s)

Recall that the Value Function is equivalent to

$$V_\pi (s) = E_\pi \left[ \left( R(t+1) + \gamma \sum_{\tau}^\infty \gamma^{\tau} R(t+2+\tau) \right) \mid s \right]  .$$

We use the general expression for $E_\pi \left[ (\text{anything}) \mid s \right]$ described in the previous subsection to make the step to

$$V_\pi (s) = \sum_a \left( \pi( a \mid s) \sum_r \sum_{s'} p(r,s' \mid s,a) \left\lbrace R(t+1) + \gamma \sum_{\tau}^\infty \gamma^{\tau} R(t+2+\tau) \right\rbrace \right)  .$$

$R(t+1)$ can just be written as $r$. Also, as $E(E(X))=E(X)$, hence $E(A+B) = E(A + E(B))$, thus $E(X) = E(E(X \mid Y))$, we have the property 

$$E\left(E\left(G(t+1) \mid (\text{anything}) \right) \right) = E\left( G(t+1) \right)  .$$

Here, we pick $\text{anything}$ to be $S_{t+1} = s'$ to correspond to the new state $s'$. This brings us to

$$V_\pi (s) = \sum_a \left( \pi( a \mid s) \sum_r \sum_{s'} p(r,s' \mid s,a) \left\lbrace r + \gamma E_\pi \left[ \sum_{\tau}^\infty \gamma^{\tau} R(t+2+\tau) \mid S_{t+1} = s' \right] \right\rbrace \right)  .$$

So, by definition of $V_\pi (s)$, this is equivalent to

$$V_\pi (s) = \sum_a \left( \pi( a \mid s) \sum_r \sum_{s'} p(r,s' \mid s,a) \left\lbrace r + \gamma E_\pi V_\pi(s') \right\rbrace \right)  .$$

This shows us formally the recursiveness of the Value Function. It is important to note that $V_\pi(s')$ encapsulates all knowledge about the future, so in order to look into the future from $s$, we only need to look at $V_\pi(s')$ - just one step ahead!

The above equation is very important in RL. All the algorithms we're going to investigate are based on this equation. This equation is named "The Bellman Equation", named after Richard Belmann (who pioneered dynamic programming, a popular technique which can help solve MDPs). The equation is efficient because it builds up using a "bottom-up" approach.

## Important Functions

### The State-Value Function

What we've just learned about is called the State-Value Function,

$$V_\pi (s) = E_\pi \left[ G(t) \mid S_t = s \right]  .$$

### The Action-Value Function

There is another value function, called the Action-Value Function,

$$Q_\pi (s,a) = E_\pi \left[ G(t) \mid S_t = s, A_t = a \right]  .$$

$Q_\pi (s,a)$ is called the Action-Value Function because the action is also a parameter of the function. For this function, the space complexity is quadratic due to having two inputs $s$ and $a$.

### Which do we use?

The value function we use depends on the algorithm that we're using. There exist some algorithms that only apply when we use the State-Value Function, and some others that only apply when we use the Action-Value Function.